In [2]:
!pip install openai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 11.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import random
import pandas as pd
from utils import file_control

## Keyword 생성
- gpt3.5로 [문서, 질의] pair 데이터를 생성하기 이전에 키워드 생성해 더 다양한 질의를 생성할 수 있도록 유도합니다.

In [52]:
load_dotenv()
API_KEY = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY

# gpt-4-turbo
# gpt-3.5-turbo-0125
llm_model = "gpt-3.5-turbo-0125"
client = OpenAI()

In [53]:
docs_df = file_control.read_jsonl("data/documents.jsonl", to_csv=True)
docs_df['index'] = docs_df.index

docs_df.to_json("data/documents_index.jsonl", orient='records', lines=True, force_ascii=False)
docs_df = file_control.read_jsonl("data/documents_index.jsonl")

In [56]:
# 샘플 프롬프트
ran = random.randint(0, len(docs_df)-1)
# ran = 0
print(ran)

augment_instruct_test = """
## Role
키워드 생성기

## Instructions
- 주어진 내용을 보고 중요한 키워드만 추출하거나 생성한다.
- 내용에는 없지만 관련성이 높은 키워드를 생성한다.
- 중요한 키워드를 앞쪽에 배치한다.
- JSON 포맷으로 키워드를 생성한다.

## Content
%s

## Output format
{"keywords": [$word1, $word2, $word3, ...]}

""" % (docs_df[ran]["content"])

print(augment_instruct_test.strip())

2541
## Role
키워드 생성기

## Instructions
- 주어진 내용을 보고 중요한 키워드만 추출하거나 생성한다.
- 내용에는 없지만 관련성이 높은 키워드를 생성한다.
- 중요한 키워드를 앞쪽에 배치한다.
- JSON 포맷으로 키워드를 생성한다.

## Content
식물 세포와 동물 세포는 구조적으로 차이가 있습니다. 식물 세포에는 동물 세포에는 없는 세포 부분이 있습니다. 이 부분은 세포 벽입니다. 세포 벽은 식물 세포의 외부를 둘러싸고 있는 견고한 구조물로, 세포의 형태를 유지하고 보호하는 역할을 합니다. 세포 벽은 주로 섬유소로 이루어져 있으며, 식물 세포의 크기와 형태를 결정하는 중요한 역할을 합니다. 동물 세포에는 세포 벽이 없기 때문에, 동물 세포는 보다 유연하고 다양한 형태를 가질 수 있습니다.

## Output format
{"keywords": [$word1, $word2, $word3, ...]}


### API 호출 테스트

In [31]:
augment_instruct = """
## Role
키워드 생성기

## Instructions
- 주어진 내용을 보고 중요한 키워드만 추출하거나 생성한다.
- 내용에는 없지만 관련성이 높은 키워드를 생성한다.
- 중요한 키워드를 앞쪽에 배치한다.
- JSON 포맷으로 키워드를 생성한다.

## Output format
{"keywords": [$word1, $word2, $word3, ...]}

"""
augment_instruct = augment_instruct.strip()

print(augment_instruct.strip())

## Role
키워드 생성기

## Instructions
- 주어진 내용을 보고 중요한 키워드만 추출하거나 생성한다.
- 내용에는 없지만 관련성이 높은 키워드를 생성한다.
- 중요한 키워드를 앞쪽에 배치한다.
- JSON 포맷으로 키워드를 생성한다.

## Output format
{"keywords": [$word1, $word2, $word3, ...]}


In [32]:
messages = [
        {"role": "system", "content": augment_instruct},
        {"role": "user", "content": docs_df[ran]["content"]}
    ]
messages

[{'role': 'system',
  'content': '## Role\n키워드 생성기\n\n## Instructions\n- 주어진 내용을 보고 중요한 키워드만 추출하거나 생성한다.\n- 내용에는 없지만 관련성이 높은 키워드를 생성한다.\n- 중요한 키워드를 앞쪽에 배치한다.\n- JSON 포맷으로 키워드를 생성한다.\n\n## Output format\n{"keywords": [$word1, $word2, $word3, ...]}'},
 {'role': 'user',
  'content': '인(원자 번호 15)은 주기율표에서 3주기 5족에 위치하고 있습니다. 이 위치를 기준으로 인은 5개의 원자가 전자를 가지고 있습니다. 이는 인의 전자 구성에 의해 결정됩니다. 인은 1s2 2s2 2p6 3s2 3p3의 전자 구성을 가지고 있으며, 이는 총 15개의 전자로 구성되어 있습니다. 이 중에서 외부 전자 껍질에는 5개의 전자가 존재하므로, 인은 5개의 원자가 전자를 가지고 있습니다.'}]

In [33]:
result = client.chat.completions.create(
            model=llm_model,
            messages=messages,
            temperature=0.7,
            response_format={"type": "json_object"},
            seed=42
)
json.loads(result.choices[0].message.content)

{'keywords': ['인', '주기율표', '3주기', '5족', '원자 번호 15', '전자 구성', '외부 전자 껍질', '전자']}

### API 호출 자동화 및 데이터 합성

In [38]:
def get_keywords(row_data):
    data = {
        "index": row_data['index'],
        "docid": row_data['docid'],
        "content": row_data['content'],
        "keywords": []
    }
    
    # print(data)
    
    messages = [
        {"role": "system", "content": augment_instruct},
        {"role": "user", "content": data['content']}
    ]
    
    # print(messages)
    
    try:
        result = client.chat.completions.create(
                    model=llm_model,
                    messages=messages,
                    temperature=0.7,
                    response_format={"type": "json_object"},
                    timeout=10,
                    seed=42
        )
        data['keywords'] = json.loads(result.choices[0].message.content)['keywords']
        
    except Exception as e:
        print(e)
        print(data)
    
    with open(f"data/gpt_data/keywords/doc_keyword{data['index']}.json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)
        
    return data

In [39]:
import concurrent
from tqdm import tqdm
from datetime import datetime
from functools import partial

In [41]:
# ========= API 호출 주의 =========
# 키워드 생성(멀티쓰레드)
with concurrent.futures.ThreadPoolExecutor(4) as executor:
    new_dataset = list(tqdm(executor.map(get_keywords, docs_df[0:10]), total=len(docs_df[0:10])))

100%|██████████| 10/10 [00:05<00:00,  1.86it/s]


### 결과 병합 및 저장

In [43]:
# ========= 덮어쓰기 주의 =========
# 경로의 모든 jsonl 데이터를 합친후 저장
file_control.conbine_json("data/gpt_data/keywords", "data/gpt_data/result/documents_keywords_test.jsonl")

In [44]:
# index 순서 맞춰서 다시 저장
keyword_df = file_control.read_jsonl("data/gpt_data/result/documents_keywords_test.jsonl", True)
keyword_df = keyword_df.sort_values(by='index').reset_index(drop=True)
keyword_df.to_json("data/gpt_data/result/documents_keywords_test.jsonl", orient='records', lines=True, force_ascii=False)

## 질문 생성
- 생성한 키워드와 문서내용을 바탕으로 질의를 생성합니다.

In [49]:
load_dotenv()
API_KEY = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY

# gpt-4-turbo
# gpt-3.5-turbo-0125
llm_model = "gpt-3.5-turbo-0125"
client = OpenAI()

In [50]:
# 생성한 키워드 불러오기
documents_keywords = file_control.read_jsonl("data/gpt_data/result/documents_keywords.jsonl", to_csv=False)

In [57]:
# 샘플 프롬프트
ran = random.randint(0, len(documents_keywords)-1)
# ran=3571
print(ran)

augment_instruct_test = """
## Role
질문 생성기

## Instructions
- 주어진 키워드를 참고하여 질문을 5개 생성해줘.
- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 생성해줘.
- 앞쪽에 위치한 키워드들은 반드시 포함하여 질문을 생성해줘.
- 최대한 다양한 질문을 생성해줘.
- 질문은 한문장으로 간결하게 구성해줘.
- 한국어로 질문을 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Keywords
%s

## Content
%s

## Output format
{"questions": [$question1, $question2, $question3, $question4, $question5]}
""" % (documents_keywords[ran]['keywords'], documents_keywords[ran]['content'])

print(augment_instruct_test)

2373

## Role
질문 생성기

## Instructions
- 주어진 키워드를 참고하여 질문을 5개 생성해줘.
- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 생성해줘.
- 앞쪽에 위치한 키워드들은 반드시 포함하여 질문을 생성해줘.
- 최대한 다양한 질문을 생성해줘.
- 질문은 한문장으로 간결하게 구성해줘.
- 한국어로 질문을 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Keywords
['환형동물', '절지동물', '마디', '신체 구조', '동물', '원형 몸', '다리', '감각기관', '먹이 섭취', '번식', '근육', '신경계', '지렁이', '지하 생활', '해양 생활', '관련성']

## Content
환형동물과 절지동물은 서로 유사한 점이 있는데 두 문의 구성원들이 마디로 나뉘어진 신체를 가지고 있다는 것이다. 환형동물은 동물의 한 종류로, 그들은 몸이 원형이며 여러 개의 마디로 이루어져 있다. 예를 들어, 지네와 같은 작은 환형동물은 몸이 여러 개의 마디로 나뉘어져 있으며, 각 마디에는 다리와 감각기관이 있다. 또한, 환형동물은 먹이를 섭취하고 번식하는 데에도 마디를 사용한다. 절지동물은 또 다른 동물의 종류로, 그들도 마디로 나뉘어진 신체를 가지고 있다. 예를 들어, 지렁이는 몸이 여러 개의 마디로 이루어져 있으며, 각 마디에는 근육과 신경계가 있다. 절지동물은 지렁이처럼 지하에서 생활하거나, 해양에서 수중 생활을 할 수도 있다. 환형동물과 절지동물은 이러한 공통점을 가지고 있으며, 이러한 특징은 그들이 서로 관련이 있다는 것을 시사한다.

## Output format
{"questions": [$question1, $question2, $question3, $question4, $question5]}



### API 호출 테스트

In [58]:
ran = random.randint(0, len(documents_keywords)-1)
# ran = 0
print(ran)

augment_instruct = """
## Role
질문 생성기

## Instructions
- 주어진 키워드를 참고하여 질문을 5개 생성해줘.
- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 생성해줘.
- 앞쪽에 위치한 키워드들은 반드시 포함하여 질문을 생성해줘.
- 최대한 다양한 질문을 생성해줘.
- 질문은 한문장으로 간결하게 구성해줘.
- 한국어로 질문을 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Output format
{"questions": [$question1, $question2, $question3, $question4, $question5]}
""" 


user_input = """
## Keywords
%s

## Content
%s
"""% (documents_keywords[ran]['keywords'], documents_keywords[ran]['content'])


print(augment_instruct)
print(user_input)

2507

## Role
질문 생성기

## Instructions
- 주어진 키워드를 참고하여 질문을 5개 생성해줘.
- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 생성해줘.
- 앞쪽에 위치한 키워드들은 반드시 포함하여 질문을 생성해줘.
- 최대한 다양한 질문을 생성해줘.
- 질문은 한문장으로 간결하게 구성해줘.
- 한국어로 질문을 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Output format
{"questions": [$question1, $question2, $question3, $question4, $question5]}


## Keywords
['염수', '담수', '해안', '염분', '밀도', '퍼져나가는 과정', '영양분', '해안 생태계']

## Content
담수와 염수가 강어귀에서 만나면, 염수는 일반적으로 담수 아래로 흐르는데 그 이유는 염수가 밀도가 더 높기 때문입니다. 이러한 현상은 해안에서 퍼져나가는 과정을 통해 발생합니다. 해안에서 염수가 담수와 만나면, 염수는 담수보다 더 높은 염분 농도를 가지고 있기 때문에 담수 아래로 흐르게 됩니다. 이러한 현상은 담수에 농축된 영양분에도 영향을 미칩니다. 염수가 담수로 흐르면서 담수에 농축된 영양분도 함께 퍼져나가게 되어, 해안 생태계에 영양분을 공급하게 됩니다. 따라서, 염수가 해안에서 퍼져나가는 것이 담수에 농축된 영양분에 가장 가능성이 높은 영향을 미치는 것으로 생각됩니다.



In [59]:
messages = [
    {"role": "system", "content": augment_instruct},
    {"role": "user", "content": user_input}
]
messages

[{'role': 'system',
  'content': '\n## Role\n질문 생성기\n\n## Instructions\n- 주어진 키워드를 참고하여 질문을 5개 생성해줘.\n- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 생성해줘.\n- 앞쪽에 위치한 키워드들은 반드시 포함하여 질문을 생성해줘.\n- 최대한 다양한 질문을 생성해줘.\n- 질문은 한문장으로 간결하게 구성해줘.\n- 한국어로 질문을 생성해줘.\n- 아래 JSON 포맷으로 생성해줘.\n\n## Output format\n{"questions": [$question1, $question2, $question3, $question4, $question5]}\n'},
 {'role': 'user',
  'content': "\n## Keywords\n['염수', '담수', '해안', '염분', '밀도', '퍼져나가는 과정', '영양분', '해안 생태계']\n\n## Content\n담수와 염수가 강어귀에서 만나면, 염수는 일반적으로 담수 아래로 흐르는데 그 이유는 염수가 밀도가 더 높기 때문입니다. 이러한 현상은 해안에서 퍼져나가는 과정을 통해 발생합니다. 해안에서 염수가 담수와 만나면, 염수는 담수보다 더 높은 염분 농도를 가지고 있기 때문에 담수 아래로 흐르게 됩니다. 이러한 현상은 담수에 농축된 영양분에도 영향을 미칩니다. 염수가 담수로 흐르면서 담수에 농축된 영양분도 함께 퍼져나가게 되어, 해안 생태계에 영양분을 공급하게 됩니다. 따라서, 염수가 해안에서 퍼져나가는 것이 담수에 농축된 영양분에 가장 가능성이 높은 영향을 미치는 것으로 생각됩니다.\n"}]

In [60]:
result = client.chat.completions.create(
            model=llm_model,
            messages=messages,
            temperature=1.0,
            response_format={"type": "json_object"},
            seed=42
)
json.loads(result.choices[0].message.content)

{'questions': ['염수와 담수가 만나는 곳에서 염수가 일반적으로 어떤 방향으로 흐르는가?',
  '해안에서 염수와 담수가 만나면 염수가 담수보다 더 높은 무엇을 가지고 있는가?',
  '해안에서 염수가 담수로 흐르면 어떤 현상이 발생하며 이는 어떠한 영향을 미치는가?',
  '염수가 담수에 농축된 영양분을 함께 퍼져나가게 되는 과정은 어디에서 발생하는가?',
  '해안 생태계에 영양분을 공급하는 과정은 무엇에 의해 이루어지는가?']}

### API 호출 자동화 및 데이터 합성

In [61]:
# 프롬프트 작성
ran = random.randint(0, len(documents_keywords)-1)
ran = 0

# == 지시사항 ==
augment_instruct = """
## Role
질문 생성기

## Instructions
- 주어진 키워드를 참고하여 질문을 5개 생성해줘.
- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 생성해줘.
- 앞쪽에 위치한 키워드들은 반드시 포함하여 질문을 생성해줘.
- 최대한 다양한 질문을 생성해줘.
- 질문은 한문장으로 간결하게 구성해줘.
- 한국어로 질문을 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Output format
{"questions": [$question1, $question2, $question3, $question4, $question5]}
""" 

# == 모델 입력 ==
user_input = """
## Keywords
%s

## Content
%s
"""% (documents_keywords[50]['keywords'], documents_keywords[50]['content'])

print(augment_instruct)
print(user_input)


## Role
질문 생성기

## Instructions
- 주어진 키워드를 참고하여 질문을 5개 생성해줘.
- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 생성해줘.
- 앞쪽에 위치한 키워드들은 반드시 포함하여 질문을 생성해줘.
- 최대한 다양한 질문을 생성해줘.
- 질문은 한문장으로 간결하게 구성해줘.
- 한국어로 질문을 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Output format
{"questions": [$question1, $question2, $question3, $question4, $question5]}


## Keywords
['물을 이용한 전기 생산', '수력 발전소', '지역 생태계', '환경적인 측면', '생태계의 균형', '수력 발전소 건설', '물의 흐름 변화', '수중 생태계 영향', '자연 환경 희생']

## Content
움직이는 물을 이용해 전기를 생산하는 것은 많은 이점이 있지만, 단점도 존재합니다. 그 중 하나는 지역 생태계가 혼란될 수 있다는 것입니다. 물을 이용한 전기 생산은 보통 대규모 수력 발전소에서 이루어지는데, 이로 인해 물의 흐름이 변화하고 수중 생태계에 영향을 줄 수 있습니다. 또한, 수력 발전소를 건설하기 위해 수많은 토지와 자연 환경을 희생해야 할 수도 있습니다. 이는 지역 생태계에 큰 영향을 미칠 수 있으며, 생태계의 균형을 깨뜨릴 수도 있습니다. 따라서, 움직이는 물을 이용해 전기를 생산하는 것은 환경적인 측면에서 신중한 고려가 필요한 분야입니다.



In [65]:
def get_questions(row_data):
    global start_point
    data = {
        "index": row_data['index'],
        "docid": row_data['docid'],
        "content": row_data['content'],
        "keywords": row_data['keywords'],
        "questions": []
    }
    # print(data)
    
    # 모델 입력
    user_input = """
## Keywords
%s

## Content
%s
"""% (data['keywords'], data['content'])

    # print(augment_instruct)
    # print(user_input)
    
    messages = [
        {"role": "system", "content": augment_instruct},
        {"role": "user", "content": user_input}
    ]
    
    # print(messages)
    
    try:
        result = client.chat.completions.create(
                    model=llm_model,
                    messages=messages,
                    temperature=1.0,
                    response_format={"type": "json_object"},
                    seed=42
        )
        data['questions'] = json.loads(result.choices[0].message.content)['questions']
        
    except Exception as e:
        print(e)
        print(data)
    
    # 파일 저장
    with open(f"data/gpt_data/questions/doc_questions{data['index']}.json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)
        
    return data

In [66]:
import concurrent
from tqdm import tqdm
from datetime import datetime
from functools import partial

In [67]:
# 멀티스레딩 호출
with concurrent.futures.ThreadPoolExecutor(4) as executor:
    new_dataset = list(tqdm(executor.map(get_questions, documents_keywords[0:10]), total=len(documents_keywords[0:10])))

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:12<00:00,  1.20s/it]


### 결과 병합

In [69]:
# ========= 덮어쓰기 주의 =========
# 경로의 모든 jsonl 데이터를 합친후 저장
file_control.conbine_json("data/gpt_data/questions", "data/gpt_data/result/documents_questions_test.jsonl")

### 병합 결과물 중 누락된 데이터 찾아보기

In [70]:
doc_questions = file_control.read_jsonl("data/gpt_data/result/documents_questions_test.jsonl", True)
doc_questions_df = pd.DataFrame(doc_questions).sort_values(by='index').reset_index(drop=True)
index_list = doc_questions_df['index'].tolist()

In [72]:
# 앞뒤 row 데이터의 index 값이 1이상 차이나는 데이터 찾기
missing_index = []
for i in range(0, len(index_list)-1):
    if index_list[i+1] - index_list[i] !=1:
         print(index_list[i]+1)
         missing_index.append(index_list[i]+1)
         
missing_index

[]

In [74]:
documents_keywords_df = file_control.read_jsonl("data/gpt_data/result/documents_keywords_test.jsonl", to_csv=True)

In [75]:
# 누락된 데이터 확인
documents_keywords_df[documents_keywords_df['index'].isin(missing_index)]

,index,docid,content,keywords


In [76]:
missing_df = documents_keywords_df[documents_keywords_df['index'].isin(missing_index)]
missings = missing_df.to_json(orient='records', lines=False, force_ascii=False)
missings = json.loads(missings)

In [77]:
# 누락된 데이터 재호출
with concurrent.futures.ThreadPoolExecutor(4) as executor:
    new_dataset = list(tqdm(executor.map(get_questions, missings), total=len(missings)))

0it [00:00, ?it/s]


In [78]:
# ========= 덮어쓰기 주의 =========
# 경로의 모든 jsonl 데이터를 합친후 저장
file_control.conbine_json("data/gpt_data/questions", "data/gpt_data/result/documents_questions_test.jsonl")

In [79]:
# index 순서 맞춰서 다시 저장
questions_df = file_control.read_jsonl("data/gpt_data/result/documents_questions_test.jsonl", True)
questions_df = questions_df.sort_values(by='index').reset_index(drop=True)0
questions_df.to_json("data/gpt_data/result/documents_keywords_test.jsonl", orient='records', lines=True, force_ascii=False)